In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import add,Input,Conv1D,Activation,Flatten,Dense
from sklearn.metrics import classification_report
from timeit import default_timer as timer

Using TensorFlow backend.


In [2]:
print(keras.__version__)
print(tf.__version__)

2.2.4
1.12.0


In [3]:
# 加载数据集
def loaddata(): 
    train_fn_0 = '../../dataset/train_0.csv'
    train_fn_1 = '../../dataset/train_1.csv'
    test_fn_0 = '../../dataset/test_0.csv'
    test_fn_1 = '../../dataset/test_1.csv'
    # 加载训练集
    train_0 = np.loadtxt(train_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    train_1 = np.loadtxt(train_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    train = np.concatenate((train_0,train_1),axis=0)
    np.random.shuffle(train)
    # 加载测试集
    test_0 = np.loadtxt(test_fn_0,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    test_1 = np.loadtxt(test_fn_1,dtype=np.float32,delimiter = ',',encoding='utf-8-sig')
    test = np.concatenate((test_0, test_1), axis=0)
    np.random.shuffle(test)
    # 整理数据维度
    x_train = train[:,0:30][:,:,np.newaxis]
    x_test = test[:,0:30][:,:,np.newaxis]
    y_train = train[:,30].astype(np.int)
    y_train = np.eye(2)[y_train]
    y_test = test[:,30].astype(np.int)
    y_test = np.eye(2)[y_test]
    print(x_train.shape)
    print(y_train.shape)
    return x_train,y_train,x_test,y_test

In [4]:

#残差块
def ResBlock(x,filters,kernel_size,dilation_rate):
    r=Conv1D(filters,kernel_size,padding='same',dilation_rate=dilation_rate,activation='relu')(x) #第一卷积
    r=Conv1D(filters,kernel_size,padding='same',dilation_rate=dilation_rate)(r) #第二卷积
    if x.shape[-1]==filters:
        shortcut=x
    else:
        shortcut=Conv1D(filters,kernel_size,padding='same')(x)  #shortcut（捷径）
    o=add([r,shortcut])
    o=Activation('relu')(o)  #激活函数
    return o


## 定义TCN模型结构
inputs=Input(shape=(30,1))
x=ResBlock(inputs,filters=32,kernel_size=3,dilation_rate=1)
x=ResBlock(x,filters=32,kernel_size=3,dilation_rate=2)
x=ResBlock(x,filters=16,kernel_size=3,dilation_rate=4)
x=Flatten()(x)
x=Dense(2,activation='softmax')(x)
model=Model(inputs=inputs,outputs=x)
#查看网络结构
model.summary()
#编译模型
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


#训练模型
train_x,train_y,test_x,test_y=loaddata()
start_time = timer()
model.fit(train_x,train_y,validation_split=0.3,batch_size=512,epochs=10)
current_time = timer()
print("train_time",(current_time-start_time)/10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 1)        0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 30, 32)       128         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 30, 32)       3104        conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 30, 32)       128         input_1[0][0]                    
__________________________________________________________________________________________________
add_1 (Add

In [5]:
#评估模型
pre=model.evaluate(test_x,test_y,batch_size=500,verbose=2)
print('test_loss:',pre[0],'- test_acc:',pre[1])

TCN_y_pred=model.predict(test_x, batch_size=500)
print(classification_report(test_y, TCN_y_pred.round(),digits=4))

test_loss: 0.02020154634075654 - test_acc: 0.9952656573085393
              precision    recall  f1-score   support

           0     0.9972    0.9962    0.9967      6872
           1     0.9902    0.9928    0.9915      2633

   micro avg     0.9953    0.9953    0.9953      9505
   macro avg     0.9937    0.9945    0.9941      9505
weighted avg     0.9953    0.9953    0.9953      9505
 samples avg     0.9953    0.9953    0.9953      9505



In [6]:
# 测试10次的运行时间
test_time_tcn = []
for i in range(10):
    start_time = timer()
    model.predict(test_x, batch_size=500)
    current_time = timer()
    test_time_tcn.append(current_time-start_time)
    print(current_time-start_time)
    i = i+1
print(test_time_tcn)
print(np.mean(test_time_tcn))

0.1914812996983528
0.19755114614963531
0.18789485096931458
0.18808511644601822
0.18701016902923584
0.19002239406108856
0.1876537799835205
0.1960877925157547
0.1879209205508232
0.1910695657134056
[0.1914812996983528, 0.19755114614963531, 0.18789485096931458, 0.18808511644601822, 0.18701016902923584, 0.19002239406108856, 0.1876537799835205, 0.1960877925157547, 0.1879209205508232, 0.1910695657134056]
0.19047770351171495
